# Pipeline

### Steps

1. Import packages and define folders
2. Choose which files to use as input
3. Instantiate functions
4. Run Program

In [5]:
# Importing other necessary packages
import glob
import pandas as pd
import os
from pathlib import Path

# Import helper functions tying everything together
from scanner_functions import move_files
from scanner_functions import ocr_pipeline

## setting folders and pdfs
folder = "/home/hennes/Internship/pdfs/" # should be folder containing pdfs of election
save_folder = '/home/hennes/Internship/constituencies/' # folder into which csvs should be saved
saved = [os.path.splitext(csv)[0] for csv in next(os.walk(save_folder))[2]]
old = '/home/hennes/Internship/old_files/' # folder into which old files are moved
old_files = [folder for folder in next(os.walk(old))[1]]
allpdf = [pdf for pdf in glob.glob(folder+'*') if pdf.endswith(".pdf")] # list with all pdfs from folder

## Choosing files (Choose one of the following options)

### input = pdfs

all pdfs that do not have a corresponding file in output folder

In [2]:
# exclude pdfs for which there is already a csv in save folder
pdflist = sorted([pdf for pdf in allpdf if pdf.split('/')[-1].split('_')[0].split('.')[0] not in
           [file.split('/')[-1].split('.')[0] for file in glob.glob(save_folder+'*')]])

pdfs as per excel sheet

In [9]:
df = pd.read_excel('/home/hennes/Downloads/Book.xlsx')

# get relevant pdf numbers

worklist = df[df['Comments'] == 'redo']['Constituency number'].tolist()

# give appropriate filename endings to items

for idx, item in enumerate(worklist):
    if len(str(item)) == 1:
        worklist[idx] = f'AC00{item}.pdf'
    if len(str(item)) == 2:
        worklist[idx] = f'AC0{item}.pdf'
    if len(str(item)) == 3:
        worklist[idx] = f'AC{item}.pdf'
        
worklist = tuple(worklist)
pdflist = [pdf for pdf in allpdf if pdf.endswith(worklist)]

### input = files

all ACs that do not have a corresponding file in output folder

In [27]:
# creates list of lists with each list containing the folders of one AC constituency
filelist = []
for e in set(sorted([folder.split('-')[1] for folder in next(os.walk(old))[1] if folder.split('-')[1] not in saved])):
    filelist.append(sorted([folder for folder in old_files if folder.split('-')[1] in e]))
filelist = sorted(filelist)

All ACs as per excel sheet

In [2]:
filelist = []
for e in set(sorted([folder.split('-')[1] for folder in next(os.walk(old))[1]])):
    filelist.append(sorted([folder for folder in old_files if folder.split('-')[1] in e]))
filelist = sorted(filelist)

In [13]:
# first get list of lists with all files

filelist = []
for e in set(sorted([folder.split('-')[1] for folder in next(os.walk(old))[1]])):
    filelist.append(sorted([folder for folder in old_files if folder.split('-')[1] in e]))
filelist = sorted(filelist)


# then filter as per excel
df = pd.read_excel('/home/hennes/Downloads/Book.xlsx')

# get relevant pdf numbers

worklist = df[df['Comments'] == 'redo OCR']['Constituency number'].tolist()

# give appropriate filename endings to items

for idx, item in enumerate(worklist):
    if len(str(item)) == 1:
        worklist[idx] = f'AC00{item}'
    if len(str(item)) == 2:
        worklist[idx] = f'AC0{item}'
    if len(str(item)) == 3:
        worklist[idx] = f'AC{item}'
        
worklist = tuple(worklist)

filelist = [l for l in filelist if l[0].split('-')[1] in worklist]

IndexError: list index out of range

## Running Program

Cleaning eventual files still in pdf folder.

In [2]:
move_files(folder, old)

NameError: name 'os' is not defined

For when input is folders from old folder (tables already extracted):

In [45]:
for l in filelist:
    [shutil.copytree(old+x, folder+x) for x in l]
    pdf = f'/{l[0].split("-")[1]}'
    print(f' \nWORKING ON {pdf.split("/")[-1]}\n ')
    stop = ocr_pipeline(pdf, from_cell=True)
    if stop:
        break

 
WORKING ON 12
 
extracted cells of 12
completed ocr of 12
working on AC017-02
working on AC017-03
working on AC017-07
working on AC017-08
working on AC017-09
working on AC017-10
working on AC017-11
working on AC017-12
working on AC017-13
working on AC017-14
working on AC017-15
Saved 02 to folder.


TypeError: 'NoneType' object is not iterable

For when input is image files from old folder:

In [4]:
pdflist = [e.split('.')[0] for e in worklist]

NameError: name 'worklist' is not defined

In [16]:
pdflist = [folder+'AC188.pdf']

In [8]:
for x in pdflist:
    imglist = [img for img in next(os.walk(old))[2] if img.startswith(os.path.split(x)[1].split('.')[0])]
    [shutil.copy(old+x, folder) for x in imglist]
    pdf = f'{folder}/{x}.pdf'
    print(f' \nWORKING ON {x}\n ')
    stop = ocr_pipeline(pdf, preprocess = False, image_conversion = False, thresh = True)
    if stop:
        break

 
WORKING ON /home/hennes/Internship/pdfs/AC226.pdf
 
extracted tables of AC226.pdf.pdf
extracted cells of AC226.pdf.pdf
completed ocr of AC226.pdf.pdf
working on AC226-01
Error tokenizing data. C error: Expected 11 fields in line 4, saw 12

will try again ignoring one more line.
working on AC226-01
working on AC226-02
working on AC226-03
working on AC226-04
working on AC226-05
working on AC226-06
working on AC226-07
working on AC226-08
working on AC226-09
working on AC226-10
working on AC226-11
working on AC226-12
working on AC226-13
working on AC226-14
working on AC226-15
working on AC226-16
working on AC226-17
working on AC226-18
working on AC226-19
working on AC226-20
working on AC226-21
working on AC226-22
working on AC226-23
working on AC226-24
working on AC226-25
working on AC226-26
working on AC226-27
working on AC226-28
working on AC226-29
Saved AC226 to folder.
 
WORKING ON /home/hennes/Internship/pdfs/AC235.pdf
 
extracted tables of AC235.pdf.pdf
extracted cells of AC235.pdf

For when input files are pdf:

In [6]:
# no options
pdflist = allpdf

In [ ]:
for x in pdflist:
    print(f' \nWORKING ON {x}\n ')
    stop = ocr_pipeline(x)
    if stop:
        break

 
WORKING ON /home/hennes/Internship/pdfs/AC255.pdf
 
